In [53]:
from __future__ import print_function

# $example on$
from pyspark.ml.feature import StringIndexer
from pyspark import SparkContext
from pyspark.sql import SQLContext


sql_sc = SQLContext(sc)

In [86]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

sql_sc = SQLContext(sc)

rdd = (sc.textFile("/tigress/arpitg/dns_data/test.csv")
       .map(lambda line: line.split(',')))
dns_data_df = rdd.toDF(["ts", "xx", "srcIP", "dstIP", "qname", "qtype", "macAddr", 
                        "subnet","host_type", "host_name", "system_type", "OS", "nic_manuf"])
dns_data_df.count()
dns_data_df.printSchema()

root
 |-- ts: string (nullable = true)
 |-- xx: string (nullable = true)
 |-- srcIP: string (nullable = true)
 |-- dstIP: string (nullable = true)
 |-- qname: string (nullable = true)
 |-- qtype: string (nullable = true)
 |-- macAddr: string (nullable = true)
 |-- subnet: string (nullable = true)
 |-- host_type: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- system_type: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- nic_manuf: string (nullable = true)



In [126]:
# Field =-based analysis
fields_to_explore = ["OS", "system_type", "nic_manuf", "qname"]
for fld in fields_to_explore:
    types = (dns_data_df
             .groupBy([fld])
             .count()
             .sort("count", ascending=False)
            )
    print(fld)
    types.show()
    cnt = types.count()
    print(cnt)

OS
+--------------------+-----+
|                  OS|count|
+--------------------+-----+
|                    |73699|
|               linux|21616|
|     ms-windows-2012| 2558|
|             unknown|  986|
|     ms-windows-2008|  712|
|         apple-macos|  171|
|             freebsd|   95|
|             solaris|   77|
|                 n/a|   48|
|          vmware-esx|   25|
|          uc-cuc-p01|    6|
|     puias-linux-6.2|    4|
|         vapornet100|    2|
|nat-oitwireless-i...|    1|
+--------------------+-----+

14
system_type
+----------------+-----+
|     system_type|count|
+----------------+-----+
|                |73701|
| virtual-machine|21626|
|         dell-pc| 3354|
|        other-pc|  960|
|             sun|  166|
|         unknown|   56|
| apple-macintosh|   49|
|             n/a|   48|
|          dec-pc|   25|
|            host|    6|
|fujitsu-rx200-s6|    4|
|           other|    4|
|    host-dynamic|    1|
+----------------+-----+

13
nic_manuf
+---------------+---

In [127]:
# identify IPs that are definitely not IoT devices
non_iot_filters = {"system_type":["virtual-machine", "dec-pc", "dell-pc", "other-pc"],
                   "OS":["solaris", "vmware-esx", "freebsd", "ms-windows-2012", "ms-windows-2008", "apple-macos"]
                  }
non_iot_filters_df = dict((k,
                           (sc.parallelize(v)
                           .map(lambda p: (p,1))
                           .toDF([k,"count"])
                           .select([k]))) for k,v in non_iot_filters.iteritems()
                         )
print(non_iot_filters_df)

{'OS': DataFrame[OS: string], 'system_type': DataFrame[system_type: string]}


In [128]:
fld_2_non_iot_IPS = {}
print("Total", dns_data_df.select(["srcIP"]).distinct().count())
for fld in non_iot_filters_df:
    tmp = (dns_data_df
           .select(["srcIP", fld])
           .distinct())
    
    non_iot_IPs = (tmp
               .join(non_iot_filters_df[fld], non_iot_filters_df[fld][fld] == tmp[fld], 'inner')
               .select("srcIP")
               .distinct()
              )
    fld_2_non_iot_IPS[fld] = non_iot_IPs
    print(fld, non_iot_IPs.count())
    


Total 5409
OS 542
system_type 618
